### Let's see if the API key works

In [1]:
import langchain
import os
from model import init_llm
from dotenv import load_dotenv
from prompts import get_generation_prompt_initial
load_dotenv()
import random
import time 

verbose = False

In [2]:
# Utility functions
# Function to randomly select subsets of varying sizes
def select_random_subsets(input_list, num_subsets):
    subsets = []
    list_length = len(input_list)
    
    for _ in range(num_subsets):
        subset_size = random.randint(1, list_length)  # Random subset size
        subset = random.sample(input_list, subset_size)
        subsets.append(subset)
    
    return subsets

def extract_dimensions_and_color(text):
    # Initialize default values for dimensions and color
    dimensions, color = None, None
    
    # Split the text into lines for easier processing
    lines = text.split('\n')
    
    for line in lines:
        # Check if the line contains the keyword 'dimensions' or 'color'
        if 'dimensions' in line.lower():
            # Extract the value after the keyword 'dimensions'
            dimensions = line.split(':', 1)[-1].strip()
        elif 'color' in line.lower():
            # Extract the value after the keyword 'color'
            color = line.split(':', 1)[-1].strip()
    
    return dimensions, color

In [3]:
llm = init_llm()

In [4]:
# put random seed

features = ['price', 'telephone number', 'color', 'dimensions']
positive = ['color', 'dimensions']
negative = ["price", "telephone number"]

In [5]:
prompt = get_generation_prompt_initial()

gen_query_chain = langchain.LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose
    )

In [7]:
random_subsets = select_random_subsets(features, 100)

for i in random_subsets:
    common_elements = set(positive).intersection(set(i))
    common_elements_list = list(common_elements)
    output = gen_query_chain.predict(features = i, positives = common_elements_list)
    dimensions, color = extract_dimensions_and_color(output)
    with open('example_file.txt', 'a') as file:
    # Write the string to the file
        file.write(output + f", [{dimensions}, {color}, {i}, {common_elements}]\n\n\n")
    time.sleep(5)
